In [3]:
# Initialize Otter
!pip install otter-grader
import otter
grader = otter.Notebook("scraping_assignment.ipynb")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 3.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.2/100.2 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.5/153.5 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.1/118.1 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.9/434.9 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 9.8 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.4/587.4 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.3/55.3 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 21.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 22.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━

In [4]:
from bs4 import BeautifulSoup
import requests
import numpy as np

## 1. Get the full names of 5 star rating books in the first page. 

Check this site [https://books.toscrape.com/catalogue/page-1.html](https://books.toscrape.com/catalogue/page-1.html).

This is just the first page, and there are exactly 50 pages like this, you can just change `page-1` to `page-2` and so on. 

For this first question, Just get the names of books from `page-1` which got 5 star rating, as a list.  Names should be full. Find `img` tag for each book by inspecting the corresponding part in page, and check for full name of the book in the `alt` attribute of `img` tag. The variable name should be `fivestar_books`

In [12]:
def get_5star_books(url):
    # scrape the webpage, and create a BeautifulSoup object
    # find star rating of the book -> filter out 5 star books
    # find img tags -> find alt attribute (title of the book)
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    books = soup.find_all('article', class_='product_pod')

    fivestar_books = []
    for book in books:
        star_rating = book.find('p', class_='star-rating Five')
        if star_rating:
            title = book.find('div', class_ = 'image_container').find('img')['alt']
            fivestar_books.append(title)

    return fivestar_books

url = 'https://books.toscrape.com/catalogue/page-1.html'
fivestar_books = get_5star_books(url)

print(fivestar_books)

['Sapiens: A Brief History of Humankind', 'Set Me Free', "Scott Pilgrim's Precious Little Life (Scott Pilgrim #1)", 'Rip it Up and Start Again']


In [13]:
grader.check("q1")

q1 results: All test cases passed!

## 2. Combined prices of All Saga books available in the chart

Now Iterate over all 50 pages available to find books from Saga series. There are Saga 1,2,3,5,6 volumes available in the site. These you can get my matching pattern from the titles. Get the total prices of those five books. Return the float value (round nearest to two decimal points) and Don't change the currency. The variable name should be `total_price`

In [30]:
import re
def calculate_price_saga(pattern):
    # scrape the webpage, and create a BeautifulSoup object
    # find all the books with the pattern in the title
    # find the price of the book -> sum the prices
    Sage_price = {}
    total_price = 0
    for page in range(1,51):
        url = f'https://books.toscrape.com/catalogue/page-{page}.html'
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        books = soup.find_all('article', class_='product_pod')
        for book in books:
            sage_volume = book.find("div", class_="image_container").find("img")["alt"]
            if pattern in sage_volume:
                price = book.find("div", class_="product_price").find("p", class_="price_color").text
                price_num = float(re.sub(r'[^\d.]', '', price))
                Sage_price[sage_volume] = price
                total_price += price_num
    # result = '£'+str(round(total_price, 2))
    return Sage_price, round(total_price, 2)

pattern = 'Saga, Volume'
Sage_price_dict, total_price = calculate_price_saga(pattern)
print(total_price)


137.86


In [31]:
grader.check("q2")

q2 results: All test cases passed!

## 3. Convert the above price to Dollars using given API

Convert `total_price` to dollars using the given API. Pass the `total_price` to `pounds` as attribute in `params` for `requests.get`. Return float value, rounding to nearest two decimal points. The variable name should be `price_in_dollars`

In [47]:
import json
api_url = 'https://currency-converter-spark-infra-105732.apps.shift.nerc.mghpcc.org/convert'
def convert_to_dollars(price):
    response = requests.get(api_url, params={'pounds': price, 'convert_to': 'dollars'})
    # response [Response] -> response.content [bytes] -> response.content.decode('utf-8') [str] -> json.loads() [dict]
    response_json = json.loads(response.content.decode('utf-8'))
    price_in_dollars = response_json['dollars']
    return round(price_in_dollars,2)
price_in_dollars = convert_to_dollars(total_price)
print(price_in_dollars)

175.08


In [48]:
grader.check("q3")

q3 results: All test cases passed!

## NOTE: Q 4, 5, & 6
Check this site https://www.boxofficemojo.com/chart/top_lifetime_gross/ . For questions 4, 5, & 6, focus on the domestic only. Don't change to international.

There are multiple pages, and you can use `params` argument in the `requests.get` to move across the pages, as discusssed in the lab

## 4. List the movies in top 200 that released in or after 2020

For this question, just focus on the first page, which has top 200 (you dont have to pass any headers or params) and get the movie names released in or after 2020, as a list. The variable name should be `movies_list`

In [86]:
def get_movies(url, req_year):
    # scrape the webpage, and create a BeautifulSoup object
    # find all the movies released in or after 2020 in the table
    # find the title of the movie -> save the title in a list
    movies_list = []
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    table = soup.find('table')
    rows = table.find_all('tr')
    for row in rows[1:]:
        year = row.find('td', class_ = 'a-text-left mojo-field-type-year').text.strip()
        if int(year) >= req_year:
            title = row.find('a').text
            movies_list.append(title)
    return movies_list

url = 'https://www.boxofficemojo.com/chart/top_lifetime_gross/'
year = 2020
movies_list = get_movies(url, year)

print(movies_list)

['Spider-Man: No Way Home', 'Top Gun: Maverick', 'Avatar: The Way of Water', 'Inside Out 2', 'Barbie', 'Deadpool & Wolverine', 'The Super Mario Bros. Movie', 'Black Panther: Wakanda Forever', 'Doctor Strange in the Multiverse of Madness', 'Spider-Man: Across the Spider-Verse', 'Jurassic World Dominion', 'Minions: The Rise of Gru', 'The Batman', 'Despicable Me 4', 'Guardians of the Galaxy Vol. 3', 'Thor: Love and Thunder', 'Oppenheimer', 'The Little Mermaid', 'Dune: Part Two', 'Twisters', 'Shang-Chi and the Legend of the Ten Rings', 'Wonka']


In [87]:
grader.check("q4")

q4 results: All test cases passed!

## 5. List the Terminator movies present in the chart


Now Iterate over all 1000 movies available to find Terminator movies in the chart. These you can get my matching pattern from the titles. Get the list of Terminator movies present, as a list. The variable name should be `terminator_movies_list` [ Hint: Just movies which has the word `terminator` present in the title (case insensitive).]

In [88]:
import re
def get_movies_pattern(pattern):
    terminator_movies_list = []
    for page_num in range(1, 6):
        page_url = f'{url}?offset={(page_num-1)*200}'
        response = requests.get(page_url)
        soup = BeautifulSoup(response.content, 'html.parser')

        # Find the table containing the movie data
        table = soup.find('table')
        for row in table.find_all('tr')[1:]:
            movie_title = row.find('a').text.strip()
            if pattern in movie_title.lower():
                terminator_movies_list.append(movie_title)
    return terminator_movies_list

pattern = 'terminator'

terminator_movies_list = get_movies_pattern(pattern)

print(terminator_movies_list)

['Terminator 2: Judgment Day', 'Terminator 3: Rise of the Machines', 'Terminator Salvation', 'Terminator Genisys']


In [89]:
grader.check("q5")

q5 results: All test cases passed!

## Question 6 

Calculate the combined gross of all 4 Avengers Movies (Domestic Only)

In [90]:
import re

def calculate_netgross_pattern(pattern):
    avengers_gross = 0
    avengers_patterns = re.compile(r'avengers', re.IGNORECASE)

    for page_num in range(1, 6):
        page_url = f'{url}?offset={(page_num - 1) * 200}'
        
        response = requests.get(page_url)
        soup = BeautifulSoup(response.content, 'html.parser')
        
        table = soup.find('table')
        
        for row in table.find_all('tr')[1:]:
            movie_name = row.find('a').text.strip()
            
            if avengers_patterns.search(movie_name):
                domestic_gross = row.find('td', class_='a-text-right mojo-field-type-money').text.strip()
                domestic_gross_value = float(re.sub(r'[^\d.]', '', domestic_gross))
                avengers_gross += domestic_gross_value

    return int(avengers_gross)

pattern = 'avengers'

avengers_gross = calculate_netgross_pattern(pattern)

print(avengers_gross)


2619552260


In [91]:
grader.check("q6")

q6 results: All test cases passed!

## Question 7. 

Get the companies belonging to "Oil and gas" industry out of the set of 50 largest companies on this [wikipedia page](https://en.wikipedia.org/wiki/List_of_largest_companies_by_revenue):  

In [93]:
def get_companies(url, industry):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    companies = []

    table = soup.find('table', class_='wikitable sortable sticky-header-multi sort-under')
    
    for row in table.find_all('tr')[2:]:
        columns = row.find_all('td')
        company_name = columns[0].text.strip()
        industry_name = columns[1].text.strip()
        if industry in industry_name:
            companies.append(company_name)

    return companies

url = 'https://en.wikipedia.org/wiki/List_of_largest_companies_by_revenue'
industry = 'Oil and gas'

companies_list = get_companies(url, industry)
print(companies_list)

['Saudi Aramco', 'China Petrochemical Corporation', 'China National Petroleum Corporation', 'ExxonMobil', 'Shell', 'TotalEnergies', 'BP', 'Chevron']


In [94]:
grader.check("q7")

q7 results: All test cases passed!